In [ ]:
import pandas as pd
import numpy as np
import pydot
import graphviz
import scipy
import matplotlib.pyplot
import tensorflow as tf
from tensorflow.keras import *
from tensorflow import keras, layers
from tensorflow.keras.layers import *

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
train_data = pd.read_csv('train.csv')
new_df = pd.read_csv('data.csv')

In [ ]:
ss = pd.read_csv('sample_submission.csv')

In [ ]:
ss.head()

In [ ]:
new_df.head()

In [ ]:
image = Input((256,256,1))

in1 = Conv2D(32,3, strides = (5,5), use_bias=False, padding='same')(image)
in1 = Conv2D(32,3, strides = (3,3), use_bias=False, padding='same')(in1)
in1 = BatchNormalization(center=True, scale=False)(in1)
in1 = Activation('relu')(in1)
in1 = MaxPooling2D((2,2), strides = (1,1), padding='same')(in1)
in1 = Conv2D(64,6, strides = (3,3), use_bias=False, padding='same')(in1)
in1 = Conv2D(64,6, strides = (3,3), use_bias=False, padding='same')(in1)
in1 = BatchNormalization(center=True, scale=False)(in1)
in1 = Activation('relu')(in1)
in1 = MaxPooling2D((2,2), strides = (1,1), padding='same')(in1)
in1 = Conv2D(128,6, strides = (3,3), use_bias=False, padding='same')(in1)
in1 = Conv2D(128,6, strides = (3,3), use_bias=False, padding='same')(in1)
in1 = BatchNormalization(center=True, scale=False)(in1)
in1 = Activation('relu')(in1)
in1 = Flatten()(in1)

out1 = Dense(100, use_bias=False)(in1)
out1 = BatchNormalization(center=True, scale=False)(out1)
out1 = Activation('relu')(out1)
out1 = Dropout(0.4)(out1)
out1 = Dense(50, use_bias=False)(out1)
out1 = BatchNormalization(center=True, scale=False)(out1)
out1 = Activation('relu')(out1)
out1 = Dropout(0.4)(out1)
out1 = Dense(15, activation='relu')(out1)
out1 = Softmax(name='class_label')(out1)

out2 = Dense(50, activation='relu')(in1)
out2 = Dropout(0.2)(out2)
out2 = Dense(12, activation='relu')(out2)
out2 = Dense(4, activation='relu', name='bbox')(out2)

model2 = tf.keras.Model(inputs=image, outputs=[out1,out2])

model2.compile(optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.01), loss={'class_label': 'binary_crossentropy', 'bbox': 'mean_squared_error'}, 
               metrics=['accuracy'])

In [ ]:
files = np.load('arrays.npz')

arrays = [files[key] for key in files]

In [ ]:
test_files = np.load('test_arrays.npz')

test_arrays = [test_files[key] for key in test_files]

In [ ]:
t_ars=[]

for x in test_arrays:
    new = np.resize(x,(256,256))
    new = np.asarray(new)
    new = keras.preprocessing.image.img_to_array(new)
    t_ars.append(new)

In [ ]:
nps=[]

for x in arrays:
    new = np.resize(x,(256,256))
    new = np.asarray(new)
    new = keras.preprocessing.image.img_to_array(new)
    nps.append(new)

In [ ]:
ids = train_data.image_id.unique()

labels = []
boxes = []
for id_ in ids:
    df = new_df[new_df.id == id_]
    class_label = df.iloc[:,2].values
    bbox = df.iloc[0,3:].values
    #label = tf.io.decode_raw(label, tf.uint8)
    #label = tf.reshape(label, label.shape)
    #label = tf.one_hot(label, 10)
    labels.append(class_label)
    boxes.append(bbox)

In [ ]:
mlb = MultiLabelBinarizer()
enc = mlb.fit_transform(labels)

In [ ]:
mlb.classes_

In [ ]:
class_labels = np.stack(enc)
bbox_bound = np.stack(boxes)
data = np.stack(nps)

t_data = np.stack(t_ars)

In [ ]:
X_train, X_test, y_lab_train, y_lab_test, y_box_train, y_box_test = train_test_split(data, 
                                                                                     class_labels, 
                                                                                     bbox_bound, 
                                                                                     test_size=0.1, 
                                                                                     shuffle=True, 
                                                                                     random_state=15)

In [ ]:
model2.summary()

In [ ]:
model2.fit(X_train, [y_lab_train, y_box_train], epochs=10, batch_size=32, validation_split=0.0)

In [ ]:
cls, b = model2.predict(X_test)

In [ ]:
def iou(tbox, pbox):
    b1_width = tbox[2] - tbox[0]
    b1_length = tbox[3] - tbox[1]
    b2_width = pbox[2] - pbox[0]
    b2_length = pbox[3] - pbox[1]
    
    b1_area = b1_length * b1_width
    b2_area = b2_length * b2_width
    
    y_int = np.fmin(tbox[1] + b1_length, pbox[1] + b2_length) - np.fmax(tbox[1], pbox[1])
    x_int = np.fmin(tbox[0] + b1_width, pbox[0] + b2_width) - np.fmax(tbox[0], pbox[0])
    
    if y_int <= 0 or x_int <= 0:
        return 0
    
    intersection = y_int * x_int
    union = b1_area + b2_area
    union = union - intersection
    
    IoU = intersection / union
    
    return IoU

In [ ]:
ious = []

for tbox, pbox in zip(y_box_test, b):
    ious.append(iou(tbox, pbox))

In [ ]:
best = []

for i in ious:
    if i >= 0.5:
        best.append(i)

In [ ]:
precision = dict()
recall = dict()
average_precision = dict()
for i in range(15):
    precision[i], recall[i], _ = precision_recall_curve(y_lab_test[:,i], cls[:,i])
    average_precision[i] = average_precision_score(y_lab_test[:, i], cls[:, i])

In [ ]:
mAP = average_precision_score(y_lab_test, cls)

In [ ]:
model2.evaluate(X_test, [y_lab_test, y_box_test])

In [ ]:
tf.keras.utils.plot_model(model2)

In [ ]:
lab_pred, bb_pred = model2.predict(t_data)

In [ ]:
metric_df = pd.DataFrame(index=range(1), columns=['model', 'epochs', 'optimizer', 'evaluated label acc.', 'evaluated bbox acc.', 'mAP'])

In [ ]:
metric_df.iloc[0,0] = 'CNN'
metric_df.iloc[0,1] = '10'
metric_df.iloc[0,2] = 'SGD, lr = 0.01'
metric_df.iloc[0,3] = '.912'
metric_df.iloc[0,4] = '.722'
metric_df.iloc[0,5] = '.178'

In [ ]:
metric_df.to_csv('metrics.csv')